In [37]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support import expected_conditions as EC

from bs4 import BeautifulSoup
import pandas as pd
import json

In [38]:
service = Service()
options = webdriver.ChromeOptions()


# options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-gpu')
options.add_argument('--ignore-certificate-errors')
options.add_argument('disable-infobars')
options.add_argument('--disable-extension')

In [40]:
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

In [41]:
URL = "https://www.carrosnaweb.com.br/fichadetalhe.asp"
CODIGO = "?codigo=2"

URL_BASE = URL + CODIGO

In [43]:
driver.get(URL_BASE)
wait = WebDriverWait(driver, 10)


# try:

#     WebDriverWait(driver, 10).until(EC.frame_to_be_available_and_switch_to_it((By.CSS_SELECTOR, "table[border='0', cellspacing='1', cellpadding=''3', width='100%]")))
    
#     print ("Page is ready!")
# except TimeoutException:
#     print ("Loading took too much time!")

In [20]:
soup = BeautifulSoup(driver.page_source, 'html.parser')

In [21]:
# with open("./html-carrosnaweb.html", 'r', encoding='utf-8') as fp:
#     soup = BeautifulSoup(fp, 'html.parser')

In [22]:
principal_table = soup \
    .find('table', 
    {
        "cellspacing":"1",
        "cellpadding":"3",
        "border":"0",
        "width":"100%"
    }, recursive=True)


tbody = principal_table.find_all('tbody')[0]
rows = tbody.find_all('tr')

info_vehicles = []
filtered_rows = []

for row in rows:
    cells = row.find_all('td')
    for cell in cells:
        
        # if not cell.has_attr({"colspan":"6" "align":"center" "bgcolor":"#ffffff"}):
        info_vehicles.append(cell.text.strip().replace('', ''))


AttributeError: 'NoneType' object has no attribute 'find_all'

In [ ]:
words_to_exclude = \
    [   "Página", 
        "Compartilhe:", 
        "As informa", 
        "Ficha T", 
        "Avaliação",
        "SUSPENSÃO",
        "FREIOS",
        "DIREÇÃO",
        "PNEUS",
        "DIMENSÕES",
        "AERODINÂMICA",
        "DESEMPENHO",
        "CONSUMO",
        "AUTONOMIA",
        "MOTOR",
        "TRANSMISSÃO",
        "Fotos",
    ]

# Criar uma nova lista sem elementos que contenham palavras da lista de exclusão
filtered_list_exclude = [item for item in info_vehicles if not any(word in item for word in words_to_exclude)]


filtered_removed_empty = list(filter(None, filtered_list_exclude))

In [ ]:
info_veichle = []
filtered_desc_labels = filtered_removed_empty[1:-1]
# Convertendo o array para um dicionário
json_dict = {filtered_desc_labels[i]: filtered_desc_labels[i+1] for i in range(0, len(filtered_desc_labels), 2)}


if "Geração" not in json_dict:
    json_dict["Geração"] = ""
if "Tuchos" not in json_dict:
    json_dict["Tuchos"] = ""
if "Variação do Comando" not in json_dict:
    json_dict["Variação do Comando"] = ""
if "Acionamento comando" not in json_dict:
    json_dict["Acionamento comando"] = ""
if "Código do Cambio" not in json_dict:
    json_dict["Código do Cambio"] = ""
if "Estepe" not in json_dict:
    json_dict["Estepe"] = ""
if "Carga útil" not in json_dict:
    json_dict["Carga útil"] = ""

# Convertendo o dicionário em uma string JSON formatada
json_string = json.dumps(json_dict, indent=2,  ensure_ascii=False)


In [ ]:
json_string

'{\n  "Ano": "2002",\n  "Preço": "R$ 11.113",\n  "Desvalorização": "8,27%",\n  "Propulsão": "Combustão",\n  "Combustível": "Gasolina",\n  "IPVA": "Isento2",\n  "Seguro": "Indisponível",\n  "Revisões": "Preços não tabelados",\n  "Procedência": "Nacional",\n  "Garantia": "1 ano",\n  "Configuração": "Hatch",\n  "Porte": "Compacto",\n  "Lugares": "5",\n  "Portas": "4",\n  "Geração": "1",\n  "Nota do Leitor": "8,8\\n\\t\xa0\\nAvalie\\n10\\n9\\n8\\n7\\n6\\n5\\n4\\n3\\n2\\n1\\n0",\n  "Índice CNW": "63,57",\n  "Ranking CNW": "14659",\n  "Instalação": "Dianteiro",\n  "Aspiração": "Natural",\n  "Disposição": "Transversal",\n  "Alimentação": "Injeção multiponto",\n  "Cilindros": "4 em linha",\n  "Comando de válvulas": "Único no cabeçote",\n  "Tuchos": "Mecânicos",\n  "Cilindrada unitária": "250 cm3",\n  "Acionamento comando": "Correia dentada",\n  "Válvulas por cilindro": "2",\n  "Diâmetro do cilindro": "70 mm",\n  "Razão de compressão": "9,5:1",\n  "Curso do pistão": "64,9 mm",\n  "Deslocamento"

In [ ]:
df_vehicles = pd.DataFrame()

In [ ]:

df_vehicles = pd.concat([df_vehicles, pd.DataFrame([info_veichle])], ignore_index=True)

In [ ]:
df_vehicles